In [1]:
import os
os.chdir('../movies')
from movieLens import MovieLens

# Load the movie Lens class
ml = MovieLens()

# Algorithm

In [2]:
from surprise import Dataset, Reader, KNNWithMeans
from surprise.model_selection import train_test_split
from surprise import accuracy
import heapq
from collections import defaultdict
from operator import itemgetter
import numpy as np
import pandas as pd

In [3]:
ratings = ml.ratings
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [4]:
# Method from the Surprise library to load the DataFrame 
# Define the Reader object to parse the dataframe
reader = Reader(rating_scale=(ratings['rating'].min(), ratings['rating'].max()))

# Load the dataframe as a ratings dataset
ratingsDataset = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

# Build the full trainset
#trainSet = ratingsDataset.build_full_trainset()
trainSet, testSet = train_test_split(ratingsDataset, test_size=0.2, random_state=42)

## User item rating matrix

Matriz en la que encontramos los ratings por usuario para cada una de las películas existentes. Una columna para los usuarios y una fila en la que se encuentran todas las películas disponibles. El valor de la celda corresponde con el rating otorgado por el usuario a la película

In [5]:
# Cosine similarity function
sim_options = {'name': 'pearson',  # alternative: cosine
               'user_based': True, # compute  similarities between users
               'min_support':5     # minimum number of common items between users
               }

model = KNNWithMeans(sim_options=sim_options)
model.fit(trainSet)
simsMatrix = model.compute_similarities()

Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.


La métrica establecida corresponde con el coeficiente de pearson, dado funciona mejor que la métrica "cosine similarity" en terminos de similitud entre usuarios porque tiene en cuenta como "puntúan" los usuarios las películas

## User similarity matrix

Comparamos usuarios para detectar aquellos con perfil similar al de referencia (al que queremos recomendar)

In [6]:
trainSet.all_users()

range(0, 610)

In [7]:
len(ratings[ratings['userId']==1])

232

In [8]:
# Reference user = user to recommend to
referenceUser = 1 

# Set the number of desired similar users
k = 10

## Look up similar users

In [9]:
def getNeighbors(referenceUser,k,trainSet,threshold=None):
    
    # Get top N similar users to our reference user
    referenceUserInnerID = trainSet.to_inner_uid(referenceUser)
    
    if threshold is None:
        similarityRow = simsMatrix[referenceUserInnerID]
        similarUsers = []
        for innerID, score in enumerate(similarityRow):
            if (innerID != referenceUserInnerID):
                similarUsers.append((innerID, score))
                
    # Alternative. Select users up to a similarity threshold
    else:
        similarUsers = [(innerID, score) for (innerID, score) in enumerate(simsMatrix[referenceUserInnerID])
                if innerID != referenceUserInnerID and score > threshold]
    
    #print(len(similarUsers))
   
    # Get top N
    # Sort the elements in decreasing order by score and select top N
    kNeighbors = heapq.nlargest(k, similarUsers, key=lambda t: t[1])
    
    return kNeighbors

In [10]:
# Option 1. No threshold
kNeighbors = getNeighbors(referenceUser,10,trainSet)
kNeighbors

[(322, 0.9359709753334591),
 (127, 0.8816752506454788),
 (402, 0.8507462787278184),
 (352, 0.8416254115301732),
 (424, 0.8236319355254453),
 (507, 0.7905694150420948),
 (74, 0.7821029549487571),
 (561, 0.7467286566323438),
 (512, 0.737043474095502),
 (589, 0.7288689868556625)]

In [11]:
# Option 2. Set a threshold value
kNeighbors2 = getNeighbors(referenceUser,10,trainSet,0.8)
kNeighbors2

[(322, 0.9359709753334591),
 (127, 0.8816752506454788),
 (402, 0.8507462787278184),
 (352, 0.8416254115301732),
 (424, 0.8236319355254453)]

Vemos que el número de perfiles similares disminuye como consecuencia de estabecer un coeficiente de similitud mínimo (threshold = 0.8)

## Candidate generation and scoring

Selecionamos las películas que podríamos recomendar en primera instancia 

### First approach

Normalizamos los ratings y multiplicamos por el coeficiente de semejanza entre el usuario elegido y el de referencia

In [12]:
# Get the stuff they rated, and add up ratings for each item, weighted by user similarity
candidates = defaultdict(float)
for similarUser in kNeighbors:
    innerID = similarUser[0]
    userSimilarityScore = similarUser[1]
    theirRatings = trainSet.ur[innerID]
    for rating in theirRatings:
        candidates[rating[0]] += (rating[1] / 5.0) * userSimilarityScore
        
# Sort the candidates by score
candidates = sorted(candidates.items(), key=lambda t: t[1], reverse=True)

### Second approach

In [13]:
# Get the stuff they rated, and add up ratings for each item, weighted by user similarity
candidates2 = defaultdict(float)
similaritySum = sum([similarity[1] for similarity in kNeighbors])
for similarUser in kNeighbors:
    innerID = similarUser[0]
    userSimilarityScore = similarUser[1] / similaritySum
    theirRatings = trainSet.ur[innerID]
    for itemID, rating in theirRatings:
        candidates2[itemID] += round(userSimilarityScore * rating,2)

# Sort the candidates by score
candidates2 = sorted(candidates2.items(), key=lambda t: t[1], reverse=True)

El primer enfoque es más sencillo pero puede estar sesgado hacia películas muy valorados, ya que sólo tiene en cuenta la suma de las valoraciones. El segundo enfoque tiene en cuenta tanto la valoración como la puntuación de similitud, lo que puede hacerlo más preciso y menos sesgado. Sin embargo, requiere más cálculos y su aplicación puede resultar más compleja.

## Candidate filtering and recommendations

Filtramos aquellas recomendaciones con un score pequeño y que ya haya visto el usuario. Para ello utilizamos un set porque únicamente nos interesa saber los items que el reference user ya ha visto, plus es un objeto eficiente para datasets largos

In [14]:
def filterRec(referenceUser,trainSet, candidates):
    
    # Get top N similar users to our reference user
    referenceUserInnerID = trainSet.to_inner_uid(referenceUser)
    
    # Build a set of movies the user has already seen
    watched = set(trainSet.ur[referenceUserInnerID])
    
    # Initialize a list to store the recommendations
    recommendations = []

    # Get top-rated items from similar users:
    pos = 0
    for itemID, ratingSum in candidates:
        if not itemID in watched:
            movieID = trainSet.to_raw_iid(itemID)
            recommendation = ml.getMovieName(int(movieID)), ratingSum
            recommendations.append(recommendation)
            pos += 1
            if (pos >= 10):
                break

    rec_movies = [rec[0] for rec in recommendations]
    return rec_movies

ratingSum represents the total similarity of the reference user to all other users who rated that item

In [15]:
# Results for the first approach
rec_movies = filterRec(referenceUser,trainSet, candidates)
rec_movies

['Star Wars: Episode VI - Return of the Jedi',
 'Usual Suspects, The',
 'Seven (a.k.a. Se7en)',
 'Forrest Gump',
 'Star Wars: Episode IV - A New Hope',
 'Matrix, The',
 'Twelve Monkeys (a.k.a. 12 Monkeys)',
 'Lord of the Rings: The Two Towers, The',
 'Firm, The',
 'Green Mile, The']

In [16]:
# Results for the second approach
rec_movies2 = filterRec(referenceUser,trainSet, candidates2)
rec_movies2

['Usual Suspects, The',
 'Star Wars: Episode VI - Return of the Jedi',
 'Seven (a.k.a. Se7en)',
 'Forrest Gump',
 'Star Wars: Episode IV - A New Hope',
 'Matrix, The',
 'Twelve Monkeys (a.k.a. 12 Monkeys)',
 'Lord of the Rings: The Two Towers, The',
 'Firm, The',
 'Green Mile, The']

In [17]:
print(rec_movies == rec_movies2)

False


Vemos que ambos métodos obtienen recomendaciones muy similares. No obstante, difieren en la última película sugerida

# Metrics

In [18]:
from metrics import evaluationMetrics
em = evaluationMetrics()

In [19]:
# Get test predictions
predtest = model.test(testSet)

# Antitest
antitest = trainSet.build_anti_testset()
predantitest = model.test(antitest)

Un antitest set es una técnica utilizada en la evaluación de sistemas de recomendación que tiene como objetivo evitar la selección aleatoria de pares usuario-ítem que puedan sesgar los resultados de la evaluación.

En lugar de seleccionar al azar los usuarios y los elementos para el conjunto de prueba, un antitest set se selecciona de manera más cuidadosa y sistemática para garantizar que no se incluyan datos que podrían conducir a una evaluación engañosa. Por ejemplo, en un antitest set, se pueden excluir aquellos usuarios y elementos que tienen pocas calificaciones o que tienen una correlación atípica con otros usuarios o elementos en el conjunto de datos.

## Métricas de precisión 

In [20]:
# Compute evaluation metrics: RMSE and MAE
print("El RMSE del modelo es de: {}".format(accuracy.rmse(predtest)))
print("El MAE del modelo es de: {}".format(accuracy.mae(predtest)))

RMSE: 0.9030
El RMSE del modelo es de: 0.9029931528241073
MAE:  0.6864
El MAE del modelo es de: 0.6864474629038703


La primera métrica que tenemos es que el recomendador estima con un error de 0.90 puntos de RMSE y 0.68 puntos de media. Intuitivamente esto quiere decir que, si se recomienda una película a un usuario con una puntuación estimada de 3.5, es razonable esperar que la puntuación que el usuario asigne a la película esté en el rango de 2.6 a 4.4, ya que la estimación del recomendador puede estar fuera en una cantidad de aproximadamente 0.90 puntos en cualquier dirección. Sin embargo, esta es una estimación aproximada y el error puede variar dependiendo del usuario y la película en cuestión.

Hay que tener en cuenta que estas métricas de precisión no son muy muy importantes ya que, ¿para qué quiero predecir estupendamente una película que he valorado con un 1 si no la debería recomendar nunca? O al contrario, ¿qué más me da predecir una película valorada con un 5 con un 4.8 o un 4.3 si la decisión al final va a ser la misma? Obviamente es importante tener una buena precisión de las predicciones de los ratings pero al final lo importante es que las recomendaciones que realicemos aporten valor.

Para esto lo primero es encontrar las top N recomendaciones del conjunto de test que estamos probando y comprobar cuales de estas son realmente relevantes.

In [21]:
# Get top N recommended movies for each user based on estimated ratings
top_10 = em.getTopN(predantitest,minimumRating = 3.5)

## Métricas de relevancia

### Precision and Recall

Se ha considerdo que las dos métricas que aportan más valor a la hora de evaluar un recomendador son Precision y Recall dado un número de recomendaciones k. Para definirlos hay que tener en cuenta lo siguiente:

* Se necesita establecer un threshold mínimo para indicar un corte entre algo que se considera positivo y algo que no lo es.
* Diremos que una valoración de un usuario es **Relevante** si la ha puntuado igual o por encima de un threshold dado.
* Una recomendación será **Valorada** dentro del top K de recomendaciones si la predicción del modelo es mayor o igual a un threshold dado.

Con estas condiciones definimos la Precision@K y el Recall@K:


\begin{equation}
\text{Precision@K} = \dfrac{\# (\text{Relevantes en K}\cap\text{Valoradas en K})}{\#\text{Valoradas en K}}
\end{equation}

\begin{equation}
\text{Recall@K} = \dfrac{\# (\text{Relevantes en K}\cap\text{Valoradas en K})}{\#\text{Relevantes}}
\end{equation}

Donde "\#" Denota el número de elementos de ese conjunto.

In [22]:
# Compute precision and recall
precisions = em.getPrecision(predtest, k=10, threshold=3.5)
recalls = em.getRecall(predtest, k=10, threshold=3.5)

Estos objetos devuelven un valor de precision y recall para cada usuario del conjunto de test en cada modelo. Para obtener un dato agregado se definen las siguientes métricas.

* **Mean Average Precision at K (MAP@K)**: Media de todas las precisiones obtenidas en el paso anterior.
* **Mean Average Recall at K (MAR@K)**: Media de todos los recalls obtenidos en el paso anterior.

In [23]:
print(f"El MAP@10 del modelo es de {np.mean(list(precisions.values()))}")
print(f"El MAR@10 del modelo es de {np.mean(list(recalls.values()))}")

El MAP@10 del modelo es de 0.7299186833203226
El MAR@10 del modelo es de 0.4667214959398828


### Normalized Discounted Cummulative Gain (NDCG)

Otro factor importante a la hora de realizar un recomendador es la posición de la recomendación, siempre debemos intentar dejar las mejores recomendaciones de los usuarios en las primeras posiciones a la hora de mostrarlas.

Definimos la ganancia acumulada de las recomendaciones como la suma de los ratings reales de las recomendaciones anteriores, es decir:

\begin{equation}
\text{Cumulative Gain@k (CG@k)} = \sum_{i = 1}^{k} \text{Rating Real}_{k} = 4 + 4.5 + 5 + 3.5 + 5 = 22
\end{equation}

Esta métrica todavía no tiene en cuenta el orden, una manera de tener en cuenta el orden es dividir cada rating por un factor. Este factor va a penalizar más fuerte conforme avanza k. Así se define el Discounted Cumulative Gain.

\begin{equation}
\text{Discounted Cumulative Gain@k (DCG@k)} = \sum_{i = 1}^{k} \dfrac{\text{Rating Real}_{k}}{\log_{2}(i+1)} = \dfrac{4.5}{\log_{2}(2)} + \dfrac{4}{\log_{2}(3)} + \dfrac{5}{\log_{2}(4)} + \dfrac{3.5}{\log_{2}(5)} + \dfrac{5}{\log_{2}(6)} = 12.96
\end{equation}

Aquí me interesa que el modelo ponga las mejores puntuaciones reales al principio, por esa razón voy dividiendo por una cantidad cada vez mayor. Como esta métrica no está acotada, conviene encontrar una manera de reducir este número a un intervalo acotado. Para ello se considera la mejor ordenación posible de los ratings anteriores.

\begin{equation}
\text{ideal Discounted Cumulative Gain@k (iDCG@k)} = \sum_{i = 1}^{k} \dfrac{\text{Rating Real ordenado}_{k}}{\log_{2}(i+1)} = \dfrac{5}{\log_{2}(2)} + \dfrac{5}{\log_{2}(3)} + \dfrac{4.5}{\log_{2}(4)} + \dfrac{4}{\log_{2}(5)} + \dfrac{3.5}{\log_{2}(6)} = 13.48
\end{equation}

De esta manera puedo saber "como de lejos" me he quedado de la mejor ordenación posible. El cociente de ambas me devuelve un valor entre 0 y 1 que indica lo "bien" que mi modelo ha ordenado las recomendaciones

\begin{equation}
\text{Normalized Discounted Cumulative Gain@k (NDCG@k)} = \dfrac{\text{DCG}}{\text{iDCG}} = \dfrac{12.96}{13.48} = 0.96
\end{equation}

Existen maneras más agresivas de penalizar los errores de posiciones en las fórmulas anteriores. También hay que tener en cuenta que se penaliza más grande es K.

Para poder implementar esta métrica tenemos que adaptar el formato de la salida de las predicciones del modelo a un formato que permita trabajar las fórmulas anteriores.

In [24]:
# Compute normalized discounted cummulative gain (NDCG)
ndcgs, mean_ndcg = em.getNDCG(predtest,10)

print(f"El valor promedio del NDCG del modelo es de: {mean_ndcg}")

El valor promedio del NDCG del modelo es de: 0.9510456458372629


Hay que tener en cuenta que esta métrica no mide la calidad de las recomendaciones, ese valor a partir de las métricas anteriores. Una vez establecidas las recomendaciones, está métrica devuelve su capacidad de ordenación siendo 1 el mayor valor posible.

Otro aspecto a tener en cuenta es que al ser escalas de 0 a 5 y ser recomendaciones relativamente buenas, los valores se quedan muy cercanos a 1, en caso de usar por ejemplo una escala de acierto-error, es decir, $\{0,1\}$, la diferencia es más notable al ordenar.

## Otras métricas de interés

El último punto de métricas que tienen interés están más relacionadas con la _variedad_ en las recomendaciones. También es importante tenerlas en cuenta a la hora de evaluar como de bueno es un recomendador. 

### Coverage
La primera de ellas que vamos a ver es la **cobertura**. La **cobertura** (coverage) mide el porcentaje de elementos distintos del catálogo que ha devuelto el modelo como recomendación al menos una vez.

In [25]:
# Compute coverage
coverage = em.getCoverage(top_10,trainSet.n_items,trainSet.all_users())
print(f"La cobertura del modelo es de: {coverage}")

La cobertura del modelo es de: 0.05432347670250896


Aquí vemos como las recomendaciones son poco variadas, en torno a un 5%. En 600 usuarios, solo se han recomendado, a lo más, unas 400 y pico de películas diferentes. Es muy poco teniendo en cuenta que el catálogo dispone de 8000 películas.

Una variante de la cobertura, es la **cobertura por usuario** (UserCoverage). Este valor indica el porcentaje de usuarios que han recibido, al menos, una recomendación donde el modelo entiende que es buena, es decir, una recomendación con una puntuación superior a un threshold dado.

In [26]:
# Compute user coverage
user_coverage = em.getUserCoverage(top_10, trainSet.n_users,4)
print(f"La cobertura por usuario del modelo es de: {user_coverage}")

La cobertura por usuario del modelo es de: 0.9967213114754099


Aquí vemos como el KNN encuentra en casi todos sus usuarios al menos una recomendación por encima de 4 en su predicción

### Novelty
Por último vamos a definir una última métrica que mide la popularidad de las recomendaciones. Esta métrica es conocida como **Novelty**. Para ello se debe establecer el concepto de popularidad dentro del conjunto de películas. En este caso se define la popularidad de las películas en función del número de valoraciones recibidas.

Definimos **Novelty** como el valor promedio del ranking de las recomendaciones ofrecidas a un usuario, es decir:

\begin{equation}
\text{Novelty} = \dfrac{\sum_{u \in U} \sum_{i \in R_u} Rank(i)}{\sum_{u \in U} |R_u|}
\end{equation}

Donde:

* $U$ es el conjunto de usuarios a los que se le ha realizado al menos una recomendación
* $R_u$ es el conjunto de recomendaciones al usuario u
* $|R_u|$ es el número total de recomendaciones realizadas al usuario u
* $Rank(i)$ es el ranking de la película i, va desde 1 hasta el total de películas del conjunto.

In [27]:
# Compute novelty
novelty = em.getNovelty(top_10,trainSet)

print(f"El valor de Novelty del modelo es de: {novelty}")

El valor de Novelty del modelo es de: 4231.016581842062


El modelo KNN incluye recomendaciones de películas que han sido pocas veces puntuadas entre los usuarios dado que su ranking promedio se encuentra en torno a la película 4231 más popular (de 8000 posibles)